**Dataset Information**
WESAD is a publicly available dataset for wearable stress and affect detection. This multimodal dataset features physiological and motion data, recorded from both a wrist- and a chest-worn device, of 15 subjects during a lab study. The following sensor modalities are included: blood volume pulse, electrocardiogram, electrodermal activity, electromyogram, respiration, body temperature, and three-axis acceleration. Moreover, the dataset bridges the gap between previous lab studies on stress and emotions, by containing three different affective states (neutral, stress, amusement). In addition, self-reports of the subjects, which were obtained using several established questionnaires, are contained in the dataset. Details can be found in the dataset's readme-file, as well as in [1].


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset
from sklearn.metrics import confusion_matrix

class WESADDataset(Dataset):
  def __init__(self,dataframe):
    self.dataframe = dataframe.drop('subject', axis=1)
    self.labes = self.dataframe['label'].values
    self.dataframe.drop('label', axis=1, inplace=True)


  def __getitem__(self,idx):
    x = self.dataframe.iloc[idx].values
    y = self.labels[idx]
    return torch.Tensor(x), y

  def __len__(self):
    return len(self.dataframe)

In [ ]:
feats =   ['BVP_mean', 'BVP_std', 'BVP_min', 'BVP_max',
           'EDA_phasic_mean', 'EDA_phasic_std', 'EDA_phasic_min', 'EDA_phasic_max', 'EDA_smna_mean',
           'EDA_smna_std', 'EDA_smna_min', 'EDA_smna_max', 'EDA_tonic_mean',
           'EDA_tonic_std', 'EDA_tonic_min', 'EDA_tonic_max', 'Resp_mean',
           'Resp_std', 'Resp_min', 'Resp_max', 'TEMP_mean', 'TEMP_std', 'TEMP_min',
           'TEMP_max', 'TEMP_slope', 'BVP_peak_freq', 'age', 'height',
           'weight','subject', 'label']
layer_1_dim = len(feats) - 2
print(layer_1_dim)
print(len(feats))

29
31


In [ ]:
def get_data_loaders(df, subject_id, train_batch_size=25, test_batch_size=5):
  train_df = df[df['subject']!= subject_id].reset_index(drop=True)
  test_df = df[df['subject']==subject_id].reset_index(drop=True)

  train_dset = WESADDataset(train_df)
  test_dset = WESADDataset(test_df)

  train_loader = torch.utils.DataLoader(train_dset, batch_size=train_batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dset, batch_size=test_batch_size)

In [ ]:
class StressNet(nn.Module):
  def __init__(self):
    super(StressNet, self).__init__()
    self.fc = nn.Sequential(
        nn.Linear(29,128),
        #nn.Dropout(0.5),
        nn.ReLU(),
        nn.Linear(128,256),
        #nn.Dropout(0.5),
        nn.ReLU(),
        nn.Linear(256,2),
        #nn.Dropout(0.5),
        nn.LogSoftmax(dim=1)
    )

    def forward(self, x):
      return self.fc(x)

In [ ]:
def train(model, optimizer, train_loader, validation_loader, num_epochs):
  history = {'train_loss': {}, 'train_acc': {}, 'valid_loss': {}, 'valid_acc':{}}

  for epoch in range(num_epochs):

    # Train:
    total = 0
    correct = 0
    trainlosses = []

    for batch_index, (images,labels) in enumerate(train_loader):

      # Send to GPU (device)
      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      images, labels = images.to(device), labels.to(device)

      #Forward pass
      outputs = model(images.float())

      # Loss
      criterion = nn.CrossEntropyLoss()
      loss = criterion(outputs, labels)

      # Backwards and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      trainlosses.append(loss.item())

      # Compute accuracy
      _, argmax = torch.max(outputs,1)
      correct += (labels == argmax).sum().item() #.mean()
      total += len(labels)

    history['train_loss'][epoch] = np.mean(trainlosses)
    history['train_acc'][epoch] = correct/total
    
    if epoch % 10 == 0:
      with torch.no_grad():

        losses = []
        total = 0
        correct = 0

        for images, labels in validation_loader:
          
          images, labels = images.to(device), labels.to(device)

          #Forward pass
          outputs = model(images.float())
          loss = criterion(outputs,labels)

          # Compute accuracy
          _, argmax = torch.max(outputs, 1)
          correct += (labels == argmax).sum().item()#.mean()
          total += len(labels)

          losses.append(loss.item())

        history['valid_acc'][epoch] = np.round(correct/total, 3)
        history['valid_loss'][epoch] = np.mean(losses)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {np.mean(losses):.4}, Acc: {correct/total:.2}')
  return history

In [ ]:
def test(model, validation_loader):
  print('Evaluating model...')

  #Test
  model.eval()

  total = 0
  correct = 0
  testlosses = []
  correct_labels = []
  predictions = []

  with torch.no_grad():
    for batch_index, (images, labels) in enumerate(validation_loader):
      #Send to GPU (device)
      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

      #Forwrd pass
      outputs = model(images.float())

      # Loss
      loss = criterion(outputs, labels)

      # Compute accuracy
      _, argmax = torch.max(outputs,1)
      correct += (labels == argmax).sum().item()#.mean()

      correct_labels.extend(labels)
      predictions.extend(argmax)

    test_loss = np.mean(testlosses)
    accuracy = np.round(correct/total, 2)
    print(f'Loss: {test_loss:.4}, Acc: {accuracy:.2}')

    y_true = [label.item() for label in correct_labels]
    y_pred = [label.item() for label in predictions]

    cm = confusion_matrix(y_true, y_pred)

    #TODO: return y true and y pred, make cm after (use ytrue/ypred)
    # return [y_true, y_pred, test_loss, accuracy]



***Start Here***

In [ ]:
df = pd.read_csv('data/m14_merged.csv', index_col = 0)
subject_id_list = df['subject'].unique()
df.head()

FileNotFoundError: ignored

In [ ]:
sample = '/content/WESAD/S10/S10_quest.csv'
df = pd.read_csv(sample)

In [ ]:
#df

,# Subj;S10;;;;;;;;;;;;;;;;;;;;;;;;;
0,# ORDER;Base;Fun;Medi 1;TSST;Medi 2;bRead;fRea...
1,# START;2.5;27.53;38.42;54.3;83.17;23.39;34.58...
2,# END;22.5;34.25;45.4;66.55;90.15;24.5;36.15;7...
3,;;;;;;;;;;;;;;;;;;;;;;;;;;
4,# PANAS;2;1;3;1;1;1;1;1;1;2;1;1;2;1;3;2;1;2;1;...
5,# PANAS;2;1;3;4;1;2;1;1;1;3;1;1;3;1;2;1;2;4;1;...
6,# PANAS;1;1;2;2;1;1;1;1;1;2;1;1;1;1;1;1;1;2;1;...
7,# PANAS;4;4;3;1;3;1;1;4;2;4;1;3;2;4;4;5;1;4;4;...
8,# PANAS;2;2;1;1;1;1;1;1;1;2;1;1;1;1;2;2;1;2;1;...
9,;;;;;;;;;;;;;;;;;;;;;;;;;;
